# Parallel Corpora for Seq2seq Generation Training

## Comments

In [2]:
test = "class MyClass: \n\t\"\"\"A simple example class\"\"\" \n\ti = 12345 # le epic comment\n\twhoa = [i + 1 for i in range(0,10)]\n\n\tdef f(self):\n\t\treturn 'hello world'"
print(test)

class MyClass: 
	"""A simple example class""" 
	i = 12345 # le epic comment
	whoa = [i + 1 for i in range(0,10)]

	def f(self):
		return 'hello world'


In [10]:
import ast
def uncomment(source):
    """ 
    Takes input code and returns code with comments stripped
    Input: code (str)
    Output: code (str)
    """
    try:
        parse = ast.unparse(ast.parse(source))
    except:
        parse = 'nan'
    return parse

In [12]:
uncomment(test)

'class MyClass:\n    """A simple example class"""\n    i = 12345\n    whoa = [i + 1 for i in range(0, 10)]\n\n    def f(self):\n        return \'hello world\''

In [29]:
import pandas as pd
bq_df = pd.read_csv("data/labeled_code/combined_data.csv")

/tmp/ipykernel_21247/170157074.py:2: DtypeWarning: Columns (83,84,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  bq_df = pd.read_csv("data/labeled_code/combined_data.csv")


In [30]:
bq_df['uncommented_content'] = bq_df['content'].apply(uncomment)

In [33]:
bq_df.to_csv("data/combined_data_uncommented.csv") #remember to tell karl to dropna

## Classes

## List Comps

In [11]:
test

'class MyClass: \n\t"""A simple example class""" \n\ti = 12345 # le epic comment\n\twhoa = [i + 1 for i in range(0,10)]\n\n\tdef f(self):\n\t\treturn \'hello world\''